# Analiza mreže javnog gradskog prevoza u Beogradu

### Opis grafa
TODO

Podešavanje okruženja:
+ pip install ipykernel
+ pip install numpy scipy networkx pandas matplotlib
+ pip install scikit-learn python-docx

Linkovi:
+ [Portal otvorenih podataka](https://data.gov.rs/sr/datasets/gtfs/)
+ [General Transit Feed Specification](https://en.wikipedia.org/wiki/GTFS)
+ [Linije javnog prevoza u Beogradu](https://sr.wikipedia.org/sr-el/%D0%9B%D0%B8%D0%BD%D0%B8%D1%98%D0%B5_%D1%98%D0%B0%D0%B2%D0%BD%D0%BE%D0%B3_%D0%BF%D1%80%D0%B5%D0%B2%D0%BE%D0%B7%D0%B0_%D1%83_%D0%91%D0%B5%D0%BE%D0%B3%D1%80%D0%B0%D0%B4%D1%83)
+ [Sekretarijat za javni prevoz](https://www.bgprevoz.rs/)

### Globalne stvari

In [4]:
# importi
import numpy as np
import scipy.stats as stats
import networkx as nx
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from itertools import chain, combinations
from sklearn.cluster import SpectralClustering
from scipy.cluster.hierarchy import dendrogram


# putanje
original_path = "../data/!original/"
clean_path = "../data/clean/"

# globalne funkcije

def series_to_single_value(series):
    return series.to_list()[0]

# TODO

### Čišćenje podataka

In [5]:
# # # Originalni podaci se čuvaju u data/!original/ folderu.
# Prečišćeni podaci se čuvaju u data/clean/ folderu.
# BgNetLWalk podaci se čuvaju u data/consolidate/ folderu.    

# TODO: izabrati koji se podaci koriste za izvršavanje
# ! Komentare koji pocinju sa uzvicnikom ili upitnikom izbaciti iz finalne verzije
df_agency = pd.read_csv(f"{original_path}/agency.csv")
df_calendar_dates = pd.read_csv(f"{original_path}/calendar_dates.csv")
df_calendar = pd.read_csv(f"{original_path}/calendar.csv")
df_fare_attributes = pd.read_csv(f"{original_path}/fare_attributes.csv")
df_fare_rules = pd.read_csv(f"{original_path}/fare_rules.csv")
df_feed_info = pd.read_csv(f"{original_path}feed_info.csv")
df_frequencies = pd.read_csv(f"{original_path}/frequencies.csv")#? Treba li da radimo u opstem slucaju i pravimo da ova tabela postoji, ili je bas poenta da je ignorisemo
df_routes = pd.read_csv(f"{original_path}/routes.csv")
df_shapes = pd.read_csv(f"{original_path}/shapes.csv")
df_stops = pd.read_csv(f"{original_path}/stops.csv")
df_stop_times = pd.read_csv(f"{original_path}/stop_times.csv")#! Nisam jos pogledao sta ne valja od dtypa
df_transfers = pd.read_csv(f"{original_path}/transfers.csv")
df_trips = pd.read_csv(f"{original_path}/trips.csv")



C:\Users\Vojin\AppData\Local\Temp\ipykernel_4744\3588308021.py:17: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stop_times = pd.read_csv(f"{original_path}/stop_times.csv")#! Nisam jos pogledao sta ne valja od dtypa


### Statistička obrada podataka [5 poena]
1) Odrediti prosečno vreme između dva polaska svake od linija. Koje su linije sa najčešćim, a koje sa najređim polascima?
2) Koje su linije sa najvećim brojem stanica, a koje stanice sa najvećim brojem linija?
3) Preračunati očekivani dnevni promet stanice kao ukupan očekivani broj stajanja linije na toj stanici, određen na osnovu planiranog rasporeda vožnje. Koje su stanice sa najvećim očekivanim dnevnim prometom?
4) Svakoj stanici dodeliti tip na osnovu vrsta prevoza koje prolaze njom (stanica autobusa, tramvaja, trolejbusa ili meštovita stanica). Koliko ima stanica svakog tipa?

In [6]:
# 1) Odrediti prosečno vreme između dva polaska svake od linija.
# Koje su linije sa najčešćim, a koje sa najređim polascima?

# TODO
df_trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,2,Zimski-Subota,1421378,Spoljašnji Krug,0,NaN,00002_shape_0,0,0
1,2,Zimski-Subota,1421379,Spoljašnji Krug,0,NaN,00002_shape_0,0,0
2,2,Zimski-Subota,1421380,Spoljašnji Krug,0,NaN,00002_shape_0,0,0
3,2,Zimski-Subota,1421381,Spoljašnji Krug,0,NaN,00002_shape_0,0,0
4,2,Zimski-Subota,1421382,Spoljašnji Krug,0,NaN,00002_shape_0,0,0
...,...,...,...,...,...,...,...,...,...
66048,90004,Zimski-Radni Dan,1400708,Kumodraž,1,NaN,90004_shape_1,0,0
66049,90004,Zimski-Radni Dan,1400709,Kumodraž,1,NaN,90004_shape_1,0,0
66050,90004,Zimski-Radni Dan,1400710,Kumodraž,1,NaN,90004_shape_1,0,0
66051,90004,Zimski-Radni Dan,1400711,Kumodraž,1,NaN,90004_shape_1,0,0


In [113]:
# 2) Koje su linije sa najvećim brojem stanica, a koje stanice sa najvećim brojem linija?

# TODO
#stops (identified by the name) with routes

#! Treba izvrsiti spajanje stanica pa iskombinovati sa ovim
def find_stops_with_most_routes():
    def stop_name_from_id(stop_id):
        return df_stops[df_stops["stop_id"] == stop_id].iloc[0]["stop_name"]

    def route_id_from_trip(trip_id):
        return df_trips[df_trips["trip_id"] == trip_id].iloc[0]["route_id"]

    def test2(trip_id):
        return df_trips[df_trips["trip_id"] == 1400710].iloc[0]["route_id"]

    def test(trip_id):
        if (trip_id % 2 == 0):
            return 7
        else:
            return 9


    test_example = df_stop_times[["stop_id", "trip_id"]].head(100) #! Kod mene je presporo da probam na celom skupu
    stop_times_with_route = test_example[["stop_id", "trip_id"]].assign(route_id = test_example["trip_id"].map(lambda x: route_id_from_trip(x)))
    #stop_times_with_route = df_stop_times[["stop_id", "trip_id"]].assign(route_id = df_stop_times["trip_id"].map(lambda x: route_id_from_trip(x)))

    stop_ids_with_num_routes= stop_times_with_route[["stop_id", "route_id"]].groupby("stop_id").count().reset_index()
    stop_names_with_num_routes = stop_ids_with_num_routes.assign(stop_name = stop_ids_with_num_routes["stop_id"].map(lambda x: stop_name_from_id(x)))

    #! Treba videti kako da preimenujem sad ovu rout_id u broj ruta
    lines_with_most_routes = stop_names_with_num_routes.loc[stop_ids_with_num_routes["route_id"] == stop_ids_with_num_routes["route_id"].max()][["stop_name", "route_id"]]
    return lines_with_most_routes

find_stops_with_most_routes()


,stop_name,route_id
23,Trg Republike,2
93,Pinosava,2


In [ ]:
# 3) Preračunati očekivani dnevni promet stanice kao ukupan očekivani broj stajanja linije na toj stanici,
# određen na osnovu planiranog rasporeda vožnje. Koje su stanice sa najvećim očekivanim dnevnim prometom?

# TODO

In [ ]:
# 4) Svakoj stanici dodeliti tip na osnovu vrsta prevoza koje prolaze njom
# (stanica autobusa, tramvaja, trolejbusa ili meštovita stanica). Koliko ima stanica svakog tipa?

# TODO

### Osnovna karakterizacija modelovanih mreža [10 poena]
5) Kolika je gustina mreže?
6) Kolike su prosečne distance u okviru mreže i dijametar mreže?
7) U kojoj meri je mreža povezana i centralizovana? Navesti broj i veličine povezanih komponenata i proceniti da li postoji gigantska komponenta.
8) Koliki je prosečni, a koliki globalni koeficijent klasterizacije mreže? Kakva je raspodela lokalnog koeficijenta klasterizacije njenih čvorova? Da li je klasterisanje izraženo ili ne? Odgovor dati upoređivanjem sa slučajno generisanim Erdos-Renyi i scale free mrežama istih dimenzija.
9) Na osnovu odgovora na pitanja 6 i 8, proceniti da li mreža iskazuje osobine malog sveta.
10) Izvršiti asortativnu analizu po stepenu čvora i dati odgovor da li je izraženo asortativno mešanje. Priložiti i vizuelizaciju.
11) Da li mreža ispoljava fenomen kluba bogatih (eng. rich club phenomenon)?
12) Analizirati stajališta na osnovu tarifne zone kojoj pripadaju. Ispitati da li BGNetL i BGNetP mreže odgovaraju modelu jezgra i periferije (core-periphery model). Smatrati da prva zona pripada jezgru, a ostale periferiji.
13) Kakva je distribucija čvorova po stepenu i da li prati power law raspodelu?

In [ ]:
# 5) Kolika je gustina mreže?

#TODO

In [ ]:
# 6) Kolike su prosečne distance u okviru mreže i dijametar mreže?

#TODO

In [ ]:
# 7) U kojoj meri je mreža povezana i centralizovana?
# Navesti broj i veličine povezanih komponenata i proceniti da li postoji gigantska komponenta.

#TODO

In [ ]:
# 8) Koliki je prosečni, a koliki globalni koeficijent klasterizacije mreže?
# Kakva je raspodela lokalnog koeficijenta klasterizacije njenih čvorova?
# Da li je klasterisanje izraženo ili ne?
# Odgovor dati upoređivanjem sa slučajno generisanim Erdos-Renyi i scale free mrežama istih dimenzija.

#TODO

In [ ]:
# 9) Na osnovu odgovora na pitanja 6 i 8, proceniti da li mreža iskazuje osobine malog sveta.

#TODO

In [ ]:
# 10) Izvršiti asortativnu analizu po stepenu čvora i dati odgovor
# da li je izraženo asortativno mešanje. Priložiti i vizuelizaciju.

#TODO

In [ ]:
# 11) Da li mreža ispoljava fenomen kluba bogatih (eng. rich club phenomenon)?

#TODO

In [ ]:
# 12) Analizirati stajališta na osnovu tarifne zone kojoj pripadaju.
# Ispitati da li BGNetL i BGNetP mreže odgovaraju modelu jezgra i periferije (core-periphery model).
# Smatrati da prva zona pripada jezgru, a ostale periferiji.

#TODO

In [ ]:
# 13) Kakva je distribucija čvorova po stepenu i da li prati power law raspodelu?

#TODO

### Analiza mera centralnosti [5 poena]
14) Sprovesti analize centralnosti po stepenu, bliskosti i relacionoj centralnosti. Dati pregled najvažnijih aktera po svakoj od njih.
15) Ko su najvažniji akteri po centralnosti po sopstvenom vektoru? Šta nam to govori o njima?
16) Na osnovu prethodna dva pitanja predložiti i konstruisati heuristiku (kompozitnu meru centralnosti) za pronalaženje najvažnijih aktera i pronaći ih. Obratiti pažnju na tip mreže koji se analizira (usmerena ili neusmerena) i, shodno tome, prilagoditi koliko različite mrežne metrike utiču na heuristiku.

In [ ]:
# 14) Sprovesti analize centralnosti po stepenu, bliskosti i relacionoj centralnosti.
# Dati pregled najvažnijih aktera po svakoj od njih.

# TODO

In [ ]:
# 15) Ko su najvažniji akteri po centralnosti po sopstvenom vektoru? Šta nam to govori o njima?

# TODO

In [ ]:
# 16) Na osnovu prethodna dva pitanja predložiti i konstruisati heuristiku (kompozitnu meru centralnosti)
# za pronalaženje najvažnijih aktera i pronaći ih. Obratiti pažnju na tip mreže koji se analizira (usmerena ili neusmerena) i,
# shodno tome, prilagoditi koliko različite mrežne metrike utiču na heuristiku.

# TODO

### Detekcija komuna Luvenskom metodom [5 poena]
17) Sprovesti klasterisanje Luvenskom metodom (maksimizacijom modularnosti) u alatu Gephi za tri različite vrednosti parametra rezolucije. Konstruisati vizuelizacije i diskutovati izbor parametra rezolucije na dobijeno klasterisanje (broj i veličina klastera).
18) Koje zajednice (komune) se mogu uočiti prilikom analize mreže, a koji akteri su ključni brokeri? Da li postoji neko objašnjenje za detektovane komune? Odgovor dati posmatrajući ne samo strukturu mreže, već i atribute preračunate u pitanjima 3 i 4.

In [ ]:
# 17) Sprovesti klasterisanje Luvenskom metodom (maksimizacijom modularnosti) u alatu Gephi za tri različite vrednosti parametra rezolucije.
# Konstruisati vizuelizacije i diskutovati izbor parametra rezolucije na dobijeno klasterisanje (broj i veličina klastera).

# TODO

In [ ]:
# 18) Koje zajednice (komune) se mogu uočiti prilikom analize mreže, a koji akteri su ključni brokeri?
# Da li postoji neko objašnjenje za detektovane komune?
# Odgovor dati posmatrajući ne samo strukturu mreže, već i atribute preračunate u pitanjima 3 i 4.

# TODO

### Detekcija komuna spektralnim klasterisanjem [10 poena]
19) Predložiti funkcije sličnosti za L i P modele koje, osim strukturnih informacija iz odgovarajućih mreža, uključuju i geografske podatke o čvorovima i konstruisati matrice sličnosti i graf Laplasijane na osnovu predloženih funkcija sličnosti.
20) Sprovesti sprektralnu analizu i proceniti potencijalne kandidate za broj komuna u mreži. Uporediti rezultat sa dendogramom konstruisanim Girvan-Newman metodom.
21) Izvršiti spektralno klasterisanje na osnovu konstruisanih funkcija sličnosti i procenjenog broja klastera. Vizuelizovati klasterisanje na mapi gradskog saobraćaja, tako što će se stanice koje su svrstane u isti klaster crtati istom bojom. Izabrati boje koje daju dobar kontrast u odnosu na mapu i boje drugih klastera.
22) Ko su akteri koji se mogu okarakterisati kao ključni brokeri (mostovi) u mreži? Šta ih čini brokerima? Porediti odgovor sa brokerima dobijenim u pitanju 17.

In [ ]:
# 19) Predložiti funkcije sličnosti za L i P modele koje, osim strukturnih informacija iz odgovarajućih mreža,
# uključuju i geografske podatke o čvorovima i konstruisati matrice sličnosti i graf Laplasijane na osnovu predloženih funkcija sličnosti.

# TODO

In [ ]:
# 20) Sprovesti sprektralnu analizu i proceniti potencijalne kandidate za broj komuna u mreži.
# Uporediti rezultat sa dendogramom konstruisanim Girvan-Newman metodom.

# TODO

In [ ]:
# 21) Izvršiti spektralno klasterisanje na osnovu konstruisanih funkcija sličnosti i procenjenog broja klastera.
# Vizuelizovati klasterisanje na mapi gradskog saobraćaja, tako što će se stanice koje su svrstane u isti klaster crtati istom bojom.
# Izabrati boje koje daju dobar kontrast u odnosu na mapu i boje drugih klastera.

# TODO

In [ ]:
# 22) Ko su akteri koji se mogu okarakterisati kao ključni brokeri (mostovi) u mreži?
# Šta ih čini brokerima?
# Porediti odgovor sa brokerima dobijenim u pitanju 17.

# TODO

### Analiza mreže BGNetLWalk [5 poena]
23) Ponoviti sve analize koje imaju smisla za BGNetLWalk mrežu. Uporediti i komentarisati dobijene rezultate sa BGNetL mrežom. Da li se uočavaju značajnije promene u mreži?

In [ ]:
# 23) Ponoviti sve analize koje imaju smisla za BGNetLWalk mrežu.
# Uporediti i komentarisati dobijene rezultate sa BGNetL mrežom.
# Da li se uočavaju značajnije promene u mreži?

# TODO: pokrenuti notebook sa drugačijim parametrima